# InSituPy demonstration - InSituExperiment

In [1]:
## The following code ensures that all functions and init files are reloaded before executions.
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
from insitupy import read_xenium
import scanpy as sc
from insitupy import InSituExperiment

In [3]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

### Load Xenium data into `InSituData` object

Now the Xenium data can be parsed by providing the data path to the `InSituPy` project folder.

In [4]:
insitupy_project = Path("demo_dataset/demo_insitupy_project")
xd = read_xenium(insitupy_project)
xd.load_all(skip="transcripts")

Loading annotations...
Loading cells...
	No alternative cells found...
Loading images...
['nuclei', 'CD20', 'HER2', 'DAPI', 'HE']
Loading regions...


### Create `InSituExperiment`

#### Method 1: Manually add `InSituData` objects

In [5]:
exp = InSituExperiment()
exp.add(
    data=xd,
    metadata={
        "slide_id": xd.slide_id,
        "sample_id": xd.sample_id,
        "patient": "A"
    }
    )

In [6]:
exp

InSituExperiment with 1 samples:
           uid slide_id    sample_id patient
0     fc313c4f  0001879  Replicate 1       A

In the same way also other datasets can be added. For demonstration purposes, we just add the same dataset again and change the metadata.

In [7]:
exp.add(
    data=xd,
    metadata={
        "slide_id": xd.slide_id,
        "sample_id": xd.sample_id,
        "patient": "A",
        "therapy": "drugB"
    }
    )

In [8]:
exp

InSituExperiment with 2 samples:
           uid slide_id    sample_id patient therapy
0     fc313c4f  0001879  Replicate 1       A     NaN
1     f4e1ca30  0001879  Replicate 1       A   drugB

#### Method 2: From config file

As config file either a csv file or an excel file can be used.

Example of a valid configuration file:

| directory         | experiment_name | date       | patient    |
|-------------------|-----------------|------------|------------|
| /path/to/dataset1 | Experiment 1    | 2023-09-01 | Patient A  |
| /path/to/dataset2 | Experiment 2    | 2023-09-02 | Patient B  |

In [9]:
exp = InSituExperiment.from_config(config_path="./demo_dataset/insituexperiment_config.csv")

In [10]:
exp

InSituExperiment with 2 samples:
           uid slide_id    sample_id patient therapy
0     cf9098f8  0001879  Replicate 1       A   drugA
1     e840b98b  0001879  Replicate 1       B   drugB

#### Method 3: From regions

We can also use regions from an `InSituData` object to split the data into separate datasets and create an `InSituExperiment` from them. This can be used to select the most interesting regions and focus on them for the analysis or to split a TMA dataset into separate datasets for each core.

In [11]:
exp = InSituExperiment.from_region(
    data=xd, region_key="TMA"
)

In [12]:
exp

InSituExperiment with 6 samples:
           uid slide_id    sample_id region_key region_name
0     68508a4d  0001879  Replicate 1        TMA         A-1
1     2c1eeb99  0001879  Replicate 1        TMA         A-2
2     8279f50c  0001879  Replicate 1        TMA         A-3
3     50e296f1  0001879  Replicate 1        TMA         B-1
4     c4a25611  0001879  Replicate 1        TMA         B-2
5     6c5ebb84  0001879  Replicate 1        TMA         B-3

In [13]:
exp.show(1)

Invalid schema for package 'ome-types', please run 'npe2 validate ome-types' to check for manifest errors.


Viewer(camera=Camera(center=(0.0, 499.90625, 499.8), zoom=0.2945552291054573, angles=(0.0, 0.0, 90.0), perspective=0.0, mouse_pan=True, mouse_zoom=True), cursor=Cursor(position=(1.0, 1.0), scaled=True, size=1, style=<CursorStyle.STANDARD: 'standard'>), dims=Dims(ndim=2, ndisplay=2, last_used=0, range=((0.0, 1000.025, 0.2125), (0.0, 999.8125, 0.2125)), current_step=(2352, 2352), order=(0, 1), axis_labels=('0', '1')), grid=GridCanvas(stride=1, shape=(-1, -1), enabled=False), layers=[<Image layer 'nuclei' at 0x1a2c158f250>, <Image layer 'CD20' at 0x1a2b703a880>, <Image layer 'HER2' at 0x1a2c6cefaf0>, <Image layer 'DAPI' at 0x1a2c6df8e20>, <Image layer 'HE' at 0x1a2c716ffa0>], help='use <2> for transform', status='Ready', tooltip=Tooltip(visible=False, text=''), theme='dark', title='0001879: Replicate 1', mouse_over_canvas=False, mouse_move_callbacks=[], mouse_drag_callbacks=[], mouse_double_click_callbacks=[], mouse_wheel_callbacks=[<function dims_scroll at 0x000001A209F70EE0>], _persiste

## Add new metadata

In the following section different scenarios for adding new metadata are shown.

In [14]:
exp1 = InSituExperiment.from_region(
    data=xd, region_key="demo_regions"
)

In [15]:
exp1

InSituExperiment with 3 samples:
           uid slide_id    sample_id    region_key region_name
0     cd7c54a2  0001879  Replicate 1  demo_regions     Region1
1     3ca52d7b  0001879  Replicate 1  demo_regions     Region2
2     c316265b  0001879  Replicate 1  demo_regions     Region3

In [16]:

exp2 = exp.copy()
exp2.append_metadata(
    new_metadata="./demo_dataset/insituexperiment_new_metadata.csv",
    by="region_name", overwrite=True
)

c:\Users\ge37voy\AppData\Local\miniconda3\envs\insitupy\lib\copy.py:230: UserWarning: Pickling a SignalInstance does not copy connected weakly referenced slots.
  y[deepcopy(key, memo)] = deepcopy(value, memo)


RuntimeError: wrapped C/C++ object of type _QtMainWindow has been deleted

In [ ]:
exp2

InSituExperiment with 6 samples:
           uid slide_id    sample_id region_name therapy organ test region_key
0     e7e43ad7  0001879  Replicate 1         A-1     NaN   NaN  NaN        NaN
1     2733efc4  0001879  Replicate 1         A-2     NaN   NaN  NaN        NaN
2     eb84b5a5  0001879  Replicate 1         A-3     NaN   NaN  NaN        NaN
3     90ea95f3  0001879  Replicate 1         B-1     NaN   NaN  NaN        NaN
4     908f5fca  0001879  Replicate 1         B-2     NaN   NaN  NaN        NaN
5     92bcbc07  0001879  Replicate 1         B-3     NaN   NaN  NaN        NaN

In [20]:

exp3 = exp.copy()
exp3.append_metadata(
    new_metadata="./demo_dataset/insituexperiment_new_metadata.csv",
    by="region_name", overwrite=False
)

In [21]:
exp3

InSituExperiment with 6 samples:
           uid slide_id    sample_id region_key region_name organ test therapy
0     68508a4d  0001879  Replicate 1        TMA         A-1   NaN  NaN     NaN
1     2c1eeb99  0001879  Replicate 1        TMA         A-2   NaN  NaN     NaN
2     8279f50c  0001879  Replicate 1        TMA         A-3   NaN  NaN     NaN
3     50e296f1  0001879  Replicate 1        TMA         B-1   NaN  NaN     NaN
4     c4a25611  0001879  Replicate 1        TMA         B-2   NaN  NaN     NaN
5     6c5ebb84  0001879  Replicate 1        TMA         B-3   NaN  NaN     NaN

In [22]:

exp4 = exp.copy()
exp4.append_metadata(
    new_metadata="./demo_dataset/insituexperiment_new_metadata2.csv",
    by="region_name", overwrite=False
)

In [23]:
exp4

InSituExperiment with 6 samples:
           uid slide_id    sample_id region_key region_name organ test therapy
0     68508a4d  0001879  Replicate 1        TMA         A-1   NaN  NaN     NaN
1     2c1eeb99  0001879  Replicate 1        TMA         A-2   NaN  NaN     NaN
2     8279f50c  0001879  Replicate 1        TMA         A-3   NaN  NaN     NaN
3     50e296f1  0001879  Replicate 1        TMA         B-1   NaN  NaN     NaN
4     c4a25611  0001879  Replicate 1        TMA         B-2   NaN  NaN     NaN
5     6c5ebb84  0001879  Replicate 1        TMA         B-3   NaN  NaN     NaN

In [24]:

exp5 = exp.copy()
exp5.append_metadata(
    new_metadata="./demo_dataset/insituexperiment_new_metadata2.csv",
    by="region_name", overwrite=True
)

In [25]:
exp5

InSituExperiment with 6 samples:
           uid slide_id    sample_id region_name therapy organ test region_key
0     68508a4d  0001879  Replicate 1         A-1     NaN   NaN  NaN        NaN
1     2c1eeb99  0001879  Replicate 1         A-2     NaN   NaN  NaN        NaN
2     8279f50c  0001879  Replicate 1         A-3     NaN   NaN  NaN        NaN
3     50e296f1  0001879  Replicate 1         B-1     NaN   NaN  NaN        NaN
4     c4a25611  0001879  Replicate 1         B-2     NaN   NaN  NaN        NaN
5     6c5ebb84  0001879  Replicate 1         B-3     NaN   NaN  NaN        NaN

## Concatenate multiple `InSituExperiment` objects

In [27]:
exp

InSituExperiment with 6 samples:
           uid slide_id    sample_id region_key region_name
0     68508a4d  0001879  Replicate 1        TMA         A-1
1     2c1eeb99  0001879  Replicate 1        TMA         A-2
2     8279f50c  0001879  Replicate 1        TMA         A-3
3     50e296f1  0001879  Replicate 1        TMA         B-1
4     c4a25611  0001879  Replicate 1        TMA         B-2
5     6c5ebb84  0001879  Replicate 1        TMA         B-3

In [28]:
exp1

InSituExperiment with 3 samples:
           uid slide_id    sample_id    region_key region_name
0     cd7c54a2  0001879  Replicate 1  demo_regions     Region1
1     3ca52d7b  0001879  Replicate 1  demo_regions     Region2
2     c316265b  0001879  Replicate 1  demo_regions     Region3

In [31]:
exp_concat = InSituExperiment.concat(
    objs={
        "exp_TMA": exp,
        "exp_demo_regions": exp1
    },
    new_col_name="name"
    )

In [32]:
exp_concat

InSituExperiment with 9 samples:
           uid slide_id    sample_id    region_key region_name              name
0     68508a4d  0001879  Replicate 1           TMA         A-1           exp_TMA
1     2c1eeb99  0001879  Replicate 1           TMA         A-2           exp_TMA
2     8279f50c  0001879  Replicate 1           TMA         A-3           exp_TMA
3     50e296f1  0001879  Replicate 1           TMA         B-1           exp_TMA
4     c4a25611  0001879  Replicate 1           TMA         B-2           exp_TMA
5     6c5ebb84  0001879  Replicate 1           TMA         B-3           exp_TMA
6     cd7c54a2  0001879  Replicate 1  demo_regions     Region1  exp_demo_regions
7     3ca52d7b  0001879  Replicate 1  demo_regions     Region2  exp_demo_regions
8     c316265b  0001879  Replicate 1  demo_regions     Region3  exp_demo_regions

In [33]:
exp_concat = InSituExperiment.concat(
    objs=[exp, exp1])

In [34]:
exp_concat

InSituExperiment with 9 samples:
           uid slide_id    sample_id    region_key region_name
0     68508a4d  0001879  Replicate 1           TMA         A-1
1     2c1eeb99  0001879  Replicate 1           TMA         A-2
2     8279f50c  0001879  Replicate 1           TMA         A-3
3     50e296f1  0001879  Replicate 1           TMA         B-1
4     c4a25611  0001879  Replicate 1           TMA         B-2
5     6c5ebb84  0001879  Replicate 1           TMA         B-3
6     cd7c54a2  0001879  Replicate 1  demo_regions     Region1
7     3ca52d7b  0001879  Replicate 1  demo_regions     Region2
8     c316265b  0001879  Replicate 1  demo_regions     Region3